#### Day 16 - B
Find the optimal path from the start point (S) to the end point (E).
Part B - Find the number of unique locations in the grid that are on optimal path from S to E.

Taking a step has a cost of 1.
Taking a step has a cost of 1000.



In [2]:
#Import Libraries and settings
import bisect
from copy import deepcopy

possible_optimals = []

settings = {
    "day": "16",
    "test_data": 0
}

In [3]:
#Load Input
def load_input(settings, initial_dir=">"):
    #Derrive input file name
    if settings["test_data"]:
        data_subdir = "test"
    else:
        data_subdir = "actual"

    data_fp = f"./../input/{data_subdir}/{settings["day"]}.txt"

    #Open and read the file
    with open(data_fp) as f:
        lines = f.read().split('\n')

    grid = []

    for idx_y, line in enumerate(lines):
        if "S" in line:
            idx_x = line.find("S")
            starting_loc = (idx_x, idx_y)
            grid.append(list(line[:idx_x] + initial_dir + line[idx_x+1:]))
        else:
            grid.append(list(line))

    return grid, starting_loc

GRID_BASE, starting_loc = load_input(settings)
GRID_WIDTH = len(GRID_BASE[0])
GRID_HEIGHT = len(GRID_BASE)

In [4]:
#Convert compass direction into a dir tuple
def translate_step(dir_g, mag=1):
    v = 0
    h = 0

    if "^" in dir_g:
        v = -mag
    elif "v" in dir_g:
        v = mag

    if ">" in dir_g:
        h = mag
    elif "<" in dir_g:
        h = -mag

    return (h, v)

#Apply a direction to a location
def apply_move(loc, dir, mag=1):
    step = translate_step(dir, mag)
    return (loc[0]+step[0], loc[1]+step[1])

In [5]:
#Function to inverse a direction
def dir_inv(dir):
    dir_inv_dict = {
        "^":"v",
        "v":"^",
        ">":"<",
        "<":">"
    }
    return dir_inv_dict[dir]

#Get all valid neighbours for a given state in the grid
def get_neighbours(grid, prev_move, history):

    loc = prev_move[0][0]
    dir = prev_move[0][1]
    base_cost = prev_move[1]

    #Get possible turns
    if dir == "^" or dir == "v":
        pos_dirs = [dir, "<", ">"]
    else:
        pos_dirs = [dir, "^", "v"]

    #Check which are already visited locations
    valid_neighbours = []
    for pos_dir in pos_dirs:

        next_loc = apply_move(loc, pos_dir)
        next_char = grid[next_loc[1]][next_loc[0]]

        #If taking a step
        if pos_dir == dir:
            move_cost = base_cost + 1
            #Check if end space
            if next_char == "E":
                next_dir = "E"
            else:
                next_dir = pos_dir
            move = (next_loc, next_dir)

        #If making a turn
        else:
            move_cost = base_cost + 1000
            move = (loc, pos_dir)
        
        #Check if direction free
        if next_char == "E":
            valid_neighbours.append((move, move_cost))
        elif next_char == ".":
            #Check the opposite move is not in history
            if (move[0], dir_inv(move[1])) not in history:
                #If in history then evaluate cost
                if move in history:
                    existing_cost = history[move]
                    if move_cost <= existing_cost:
                        valid_neighbours.append((move, move_cost))
                #Otherwise add the move
                else:
                    valid_neighbours.append((move, move_cost))

    return valid_neighbours

#Consider all moves through the grid until the end is found
def grid_walker(grid, loc, dir=">"):

    #Maintain history of previous moves
    history = {
        (loc, dir):0
    }

    #Initial move
    queue = get_neighbours(grid, ((loc, dir), 0), history)

    while queue:

        #Process move
        move, cost = queue[0]
        queue.pop(0)

        move_loc = move[0]
        move_dir = move[1]

        #Add to history
        if move not in history:
            history[move] = cost

        #Check if end condition found
        if move_dir == "E":
            return move_loc, cost, history

        #Get new items for the queue
        new_items = get_neighbours(grid, (move, cost), history)

        for ni in new_items:
                bisect.insort(queue, ni, key=lambda x:x[1])

In [6]:
end_loc, cost, history = grid_walker(GRID_BASE, starting_loc)
#Solution for part A
print(cost)

115500


In [7]:
#Moves have a loc and dir and map to a cost
def possible_move(move, history, end_loc):
    move_cost = history[move]
    move_loc = move[0]
    move_dir = move[1]

    #If the location is the end location, return True to avoid trimming
    if move_loc == end_loc:
        return True

    #Check if moving in the same direction is an option
    step_loc = apply_move(move_loc, move_dir)
    step_move = (step_loc, move_dir)
    if step_move in history.keys():
        step_cost = history[step_move]
        if move_cost == step_cost - 1:
            return True
    #Edge case for the end location as it doesn't have a direction
    elif step_loc == end_loc:
        step_cost = history[(end_loc, "E")]
        if move_cost == step_cost - 1:
            return True

    
    #Check if changing direction is an option
    dirs = ["^", "<", "v", ">"]
    for pos_dir in dirs:
        if pos_dir != move_dir:
            if (move_loc, pos_dir) in history.keys():
                if move_cost == history[(move_loc, pos_dir)] - 1000:
                    return True
    
    return False

#Process the history object and remove all moves that do not lead to the end space
def refine_history(base_history, end_loc):
    #Create a working version of the history to process
    trimmed = deepcopy(base_history)

    #Set up while loop to continue until an interation where no moves are trimmed
    changing = True
    while changing:
        changing = False
        to_trim = []

        #Consider all moves remaining
        for move in trimmed.keys():
            if not possible_move(move, trimmed, end_loc):
                changing = True
                to_trim.append(move)

        #Remove all deadends
        for dead_move in to_trim:
            del trimmed[dead_move]

        #To report progress
        print(len(trimmed.keys()))

    #print(trimmed)
    return trimmed

In [8]:
sol_only = refine_history(history, end_loc)

12449
11599
10828
10239
9721
9248
8870
8509
8192
7910
7645
7412
7200
7000
6814
6634
6466
6308
6159
6015
5879
5752
5632
5518
5415
5315
5216
5121
5036
4953
4870
4786
4704
4627
4553
4484
4419
4357
4298
4239
4183
4127
4073
4021
3969
3921
3874
3830
3788
3746
3705
3664
3624
3584
3544
3507
3471
3435
3400
3366
3333
3301
3269
3238
3207
3177
3147
3119
3093
3067
3040
3013
2986
2960
2934
2908
2883
2858
2833
2809
2785
2761
2739
2717
2695
2673
2652
2632
2612
2592
2573
2554
2535
2517
2499
2482
2466
2450
2434
2418
2403
2389
2375
2361
2348
2335
2322
2309
2297
2285
2273
2261
2249
2237
2225
2213
2201
2189
2177
2165
2153
2141
2129
2117
2106
2095
2084
2073
2062
2052
2042
2032
2022
2013
2004
1995
1986
1977
1968
1959
1951
1943
1935
1927
1919
1911
1903
1895
1887
1879
1871
1864
1857
1850
1843
1836
1830
1824
1818
1812
1806
1800
1794
1788
1782
1776
1770
1764
1758
1752
1746
1740
1734
1728
1722
1716
1710
1704
1698
1692
1686
1680
1674
1668
1662
1656
1650
1644
1638
1633
1628
1623
1618
1613
1608
1602
1596
1590
1584
1

In [9]:
#Get unique locations by adding all locations leading to the solution into a set
def unique_locs(sols):
    locs = set()
    for key in sols.keys():
        locs.add(key[0])

    return locs

sol_locs = unique_locs(sol_only)

In [10]:
#Number of unique locations
print(len(sol_locs))

679


In [11]:
grid_sol = deepcopy(GRID_BASE)

for space in sol_locs:
    grid_sol[space[1]][space[0]] = "O"

for line in grid_sol:
    print("".join(line))

#############################################################################################################################################
#.#.........#.#.....#.....#...#...................#.........#.......#.....#.......#.........#...#.............#.............#.........#OOO#O#
#.#.#.#####.#.#.#.###.#.#.#.#.###.###.#########.#.#######.#.###.###.#.###.#.###.#.#.#.#####.#.#.#.#####.###.###.#########.#.#.#.#####.#O#O#O#
#.#.#.....#...#.#.....#.#...#.......#...#...#...#.#.......#.....#.#...#.#.#.#...#.#.#.....#...#.#.#.....................#.#.#.#.#...#.#O#OOO#
#.#.#####.###.#.#######.#####.###.#.###.#.#.#.###.#.#############.#####.#.#.#.###.#######.#######.#.###.#.#.#.#######.#.#.###.#.#.#.#.#O###.#
#...#.....#.......#...#.......#...#.#...#.#...#.#.#.#...#.....#.....#...............#...#.#.......#...#.#.#.#.....#...#.#.....#.#.#.#.#OOO#.#
#.###.#.###.#.#.###.#.#######.#.#.#.#.###.#####.#.#.#.#.###.#.#.#.#.###.#.###.#.###.#.#.#.#.#####.#####.#.###.###.#.#.#########.#.#.#.###O###
#.#.#.